In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
import os
import time
import sys
import numpy as np
sys.path.append('/home/chaofan/powerknowledge/data')
# sys.path.append('data/')
from read_PLAID_data import read_processed_data,get_feature_name

In [3]:
start_reading_time = time.time()
feature_select = [
    'i_mean', 'i_wave_factor', 'i_pp_rms', 'i_thd', 'pure_thd', 'P', 'Q',
    'P_F', 'i_hp1', 'z_hp1', 'i_hm2', 'z_hm2', 'i_hp2', 'z_hp2', 'i_hm3',
    'z_hm3', 'i_hp3', 'z_hp3', 'i_hm4', 'z_hm4', 'i_hp4', 'z_hp4', 'i_hm5',
    'z_hm5', 'i_hp5', 'z_hp5', 'i_hm6', 'z_hm6', 'i_hp6', 'z_hp6', 'i_hm7',
    'z_hm7', 'i_hp7', 'z_hp7'
]
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 20.538s


In [9]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(x_mh_train, y_mh_train)
print(rf0.oob_score_)
print(rf0.score(x_mh_validation, y_mh_validation))


0.9992424242424243
0.8346341463414634


In [5]:
best_score = 0.0
for n_estimator in range(10, 201, 10):
    estimator = RandomForestClassifier(n_estimators=n_estimator,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        max_depth=8,
                                        max_features='sqrt',
                                        random_state=10)
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_validation, y_mh_validation)
    if score > best_score:
        best_score = score
        best_n_estimator = n_estimator
gbc0 = RandomForestClassifier(n_estimators=best_n_estimator,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=8,
                                max_features='sqrt',
                                random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_n_estimator: %02d' % best_n_estimator)

/home/chaofan/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/chaofan/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
/home/chaofan/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/chaofan/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
train best score: 0.797073
Accuracy : 0.8021
AUC Score (train): 0.999957
AUC Score (test): 0.988156
best_n_estimator: 200


In [2]:
# test zengj
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 6.358s


In [3]:
gbc0 = RandomForestClassifier(n_estimators=200,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=8,
                                max_features='sqrt',
                                random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))

print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))


Accuracy : 0.8083
AUC Score (test): 0.979803


In [2]:
# test diandongche
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/HIOKI/try')

x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=100,
    feature_select=feature_select,
    source='HIOKI/try',
    source_json='/home/chaofan/powerknowledge/data/source/HIOKI/try.json')

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    direaction=0,
    offset=0,
    each_lenth=120,
    feature_select=feature_select,
    source='HIOKI/powerfingerprint_process',
    source_json='/home/chaofan/powerknowledge/data/source/HIOKI/powerfingerprint.json')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 1.459s


In [4]:
best_score = 0.0
for n_estimator in range(10, 201, 10):
    estimator = RandomForestClassifier(n_estimators=n_estimator,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        max_depth=8,
                                        max_features='sqrt',
                                        random_state=10)
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_train, y_mh_train)
    if score > best_score:
        best_score = score
        best_n_estimator = n_estimator
gbc0 = RandomForestClassifier(n_estimators=best_n_estimator,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=8,
                                max_features='sqrt',
                                random_state=10)
gbc0.fit(x_mh_train, y_mh_train)

y_test_pred = gbc0.predict(x_mh_test)

print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print('best_n_estimator: %02d' % best_n_estimator)

train best score: 1.000000
Accuracy : 0.95
best_n_estimator: 20
